In [3]:
import random
import copy
import multiprocessing
import pandas as pd
import numpy as np
from deap import base
from deap import creator
from deap import tools
from datetime import datetime, timedelta

In [4]:
# Load files
path = '../Data/'
festival = 'IFEMA_Festival_Videojuegos_1/'
preferences = 'Preferencias/'
file_name= 'IFEMA_Festival_Videojuegos_v_5.csv'
EVENTO = pd.read_csv(path+festival+file_name)
file_name='PREFERENCIAS_US1.csv'
PREFERENCIAS = pd.read_csv(path+festival+preferences+file_name)
PREFERENCIAS = PREFERENCIAS.sort_values('Id_Recurso')
PREFERENCIAS.head(15)

,Id_Recurso,Preferencia
101,0,0.233295
88,1,0.358799
111,2,0.120212
102,3,0.227418
113,4,0.102221
107,5,0.184035
27,6,0.494820
114,7,0.044136
50,8,0.425795
89,9,0.358799


In [5]:
# Transform to date and hour formats
EVENTO['Hora_apertura_1'] = pd.to_datetime(EVENTO['Hora_apertura_1'], format='%H:%M')
EVENTO['Hora_cierre_1'] = pd.to_datetime(EVENTO['Hora_cierre_1'], format='%H:%M')
EVENTO['Fecha_1'] = pd.to_datetime(EVENTO['Fecha_1'], format='%d/%m/%Y')

EVENTO['Hora_apertura_2'] = pd.to_datetime(EVENTO['Hora_apertura_2'], format='%H:%M')
EVENTO['Hora_cierre_2'] = pd.to_datetime(EVENTO['Hora_cierre_2'], format='%H:%M')
EVENTO['Fecha_2'] = pd.to_datetime(EVENTO['Fecha_2'], format='%d/%m/%Y')

In [6]:
# Get number of days of the event
DAYS_EVENT = (max(EVENTO.Fecha_2) - min(EVENTO.Fecha_1)).days + 1

In [8]:
# Size of the individual
SIZE_INDIVIDUAL = EVENTO.shape[0]

# Number of generations
NUM_GENERATIONS = 20

# Optimal value to obtain (not reached)
MAX_VALUE = SIZE_INDIVIDUAL + 10 * SIZE_INDIVIDUAL

# Poblation members
SIZE_POBLATION = 1000

# CXPB  is the probability with which two individuals are crossed
CXPB = 0.5

# MUTPB is the probability for mutating an individual
MUTPB = 0.2

In [9]:
# Define the fitness function for maximizing the result
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# Creation of the individual with the fitness function
creator.create("Individual", list, fitness=creator.FitnessMax)

In [10]:
# Create a toolbox for the genetic algorithm
toolbox = base.Toolbox()

# Attribute generator having only 0, 1 or N days of the current event in the body of the individual (no day, first day or N day)
toolbox.register("attr_int", random.randint, 0, DAYS_EVENT)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, SIZE_INDIVIDUAL)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [11]:
# Enable multiprocessing
pool = multiprocessing.Pool()
toolbox.register("map", pool.map)

In [12]:
# Function to find overlapped hours between resources
def find_hour_overlapping(ha1, hc1, ha2, hc2):
    found = False
    if (ha1 == ha2) or (ha2 < ha1 < hc2) or (ha1 < ha2 < hc1) or (ha2 < ha1 < hc1 < hc2) \
        or (ha1 < ha2 < hc2 < hc1):
        found = True
    return found

In [13]:
# Function to find overlapped hours in the first day
def find_first_time_overlapping(i, j):
    found = False
    
    # Get open time for the first resource
    ha1 = EVENTO.Hora_apertura_1[EVENTO.index[i]]

    # Get close time for the first resource
    hc1 = EVENTO.Hora_cierre_1[EVENTO.index[i]]

    # Get open time for the second resource
    ha2 = EVENTO.Hora_apertura_1[EVENTO.index[j]]

     # Get close time for the second resource
    hc2 = EVENTO.Hora_cierre_1[EVENTO.index[j]]
                        
    if int(ha1.hour) == 0 and int(hc1.hour) == 0:
        found = True
    else:
        found = find_hour_overlapping(ha1, hc1, ha2, hc2)
    return found

In [14]:
def find_second_time_overlapping(i, j):
    found = False
    
    # Get open time for the first resource
    ha1 = EVENTO.Hora_apertura_2[EVENTO.index[i]]

    # Get close time for the first resource
    hc1 = EVENTO.Hora_cierre_2[EVENTO.index[i]]

    # Get open time for the second resource
    ha2 = EVENTO.Hora_apertura_2[EVENTO.index[j]]

    # Get close time for the second resource
    hc2 = EVENTO.Hora_cierre_2[EVENTO.index[j]]
        
    # Exclude resources that are not active in the second 
    if int(ha1.hour) == 0 and int(hc1.hour) == 0:
        found = True
    else:
        found = find_hour_overlapping(ha1, hc1, ha2, hc2)
    return found

In [15]:
def find_mixed_time_overlapping(i, j):
    found = False
    
    # Get open time for the first resource
    ha1 = EVENTO.Hora_apertura_1[EVENTO.index[i]]

    # Get close time for the first resource
    hc1 = EVENTO.Hora_cierre_1[EVENTO.index[i]]

    # Get open time for the second resource
    ha2 = EVENTO.Hora_apertura_2[EVENTO.index[j]]

    # Get close time for the second resource
    hc2 = EVENTO.Hora_cierre_2[EVENTO.index[j]]
    
    # Exclude resources that are not active in the second 
    if int(ha1.hour) == 0 and int(hc1.hour) == 0:
        found = True
    else:    
        found = find_hour_overlapping(ha1, hc1, ha2, hc2)
    return found

In [16]:
# Function to find overlapped hours in the second day
def find_time_overlapping(i, j):
    found = False
    if find_first_time_overlapping(i, j) or find_second_time_overlapping(i, j) or find_mixed_time_overlapping(i, j):
        found = True
    return found

In [17]:
# Overlapping function to detect time problems in resources
def find_overlapping(individual):
    found = False
    day = 1
    while day <= DAYS_EVENT and not found:
        i = 0
        while i < len(individual) and not found:
            # If the resource is selected for the algorithm for day N
            if individual[i] == day:
                j = 0
                while j < len(individual) and not found:
                    if i != j:
                        if individual[j] == day:
                            found = find_time_overlapping(i, j)
                    j = j + 1
            i = i + 1
        day = day + 1
    return found

In [18]:
# Fitness function
def evalOneMax(individual):
    summa = 0
    if not find_overlapping(individual):
        for i in range(0, len(individual)):
            if individual[i] != 0:
                summa = 1 + summa + PREFERENCIAS[PREFERENCIAS['Id_Recurso'] == EVENTO.index[i]].Preferencia.values[0]
    return summa,

In [19]:
# Record the allowed operations
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=2, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [20]:
# Initial population
pop = toolbox.population(n=SIZE_POBLATION)
for i in range(0, SIZE_POBLATION):
    pop[i] =  creator.Individual(SIZE_INDIVIDUAL * [0])

In [21]:
# Evaluate the entire population
fitnesses = list(map(toolbox.evaluate, pop))

In [22]:
# Store the values of the fitness function for each individual
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

In [23]:
# Function for calculating the stats each generation
def calculate_statistics(fits, g=0):
    print("-- Generation %i --" % g)   
    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5
    print("  Min %s" % min(fits))
    print("  Max %s" % max(fits))
    print("  Avg %s" % mean)
    print("  Std %s" % std)

In [24]:
# Gather all the fitnesses in one list and print the stats
fits = [ind.fitness.values[0] for ind in pop]
calculate_statistics(fits)

-- Generation 0 --
  Min 0.0
  Max 0.0
  Avg 0.0
  Std 0.0


In [25]:
# Variable keeping track of the number of generations
g = 0

# Begin the evolution
while max(fits) < MAX_VALUE and g < NUM_GENERATIONS:
    # A new generation
    g = g + 1
    
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))
    
    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values
            
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
        
    pop[:] = offspring
    # Gather all the fitnesses in one list and print the stats
    fits = [ind.fitness.values[0] for ind in pop]
    calculate_statistics(fits, g)

-- Generation 1 --
  Min 0.0
  Max 4.477684427614696
  Avg 0.06757567692364129
  Std 0.424165076699332
-- Generation 2 --
  Min 0.0
  Max 5.206977552941924
  Avg 0.27862881539223605
  Std 0.8042834211358976
-- Generation 3 --
  Min 0.0
  Max 4.833833649973584
  Avg 0.7093742109378584
  Std 1.1364091254208195
-- Generation 4 --
  Min 0.0
  Max 4.833833649973584
  Avg 1.1490900121317795
  Std 1.339529049045089
-- Generation 5 --
  Min 0.0
  Max 5.720477231524283
  Avg 1.703734660853487
  Std 1.5372506932628742
-- Generation 6 --
  Min 0.0
  Max 7.736355918484263
  Avg 2.129487485295854
  Std 1.7382823671070569
-- Generation 7 --
  Min 0.0
  Max 7.736355918484263
  Avg 2.4514409540564843
  Std 1.93586326472465
-- Generation 8 --
  Min 0.0
  Max 7.800348722834434
  Avg 2.761771137008661
  Std 2.1554799622388594
-- Generation 9 --
  Min 0.0
  Max 10.090744046689691
  Avg 3.3025790148391243
  Std 2.3442218609117216
-- Generation 10 --
  Min 0.0
  Max 10.090744046689691
  Avg 3.67247848355815

In [26]:
# Find and print best individual
best_index = fits.index(max(fits))
best_individual = pop[best_index]
print(best_individual)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [27]:
# Print the selected resources by the genetic algorithm for the two days
for i in range(0,len(best_partial_individual)):
    if best_partial_individual[i] != 0:
        print(EVENTO.Código[EVENTO.index[i]], best_partial_individual[i], EVENTO.Hora_apertura_1[EVENTO.index[i]], EVENTO.Hora_cierre_1[EVENTO.index[i]], EVENTO.Hora_apertura_2[EVENTO.index[i]], EVENTO.Hora_cierre_2[EVENTO.index[i]])

NameError: name 'best_partial_individual' is not defined